In [1]:
%pwd

'd:\\software_3\\Generative_models\\Text_models\\chat_gpt2\\SFT_LoRA_QLoRA_RLHF'

In [2]:
import os
%pwd
print(f"The current path is changed to")
os.chdir("../")
%pwd

The current path is changed to


'd:\\software_3\\Generative_models\\Text_models\\chat_gpt2'

# Supervised Finetunning

This notebook implements the supervised funetunning which makes the pretraind model (The model which is trained on large corpus) to be able to responds to the user asked queries.

In [ ]:
import time
import torch
from torch.utils.data import Dataset, DataLoader
from gpt import GPTModel
from utils.generate import generate
from utils.download_dataset import download_and_load_dataset, partition_data
from utils.train import train_model
from utils.load_and_save_models import load_model, save_model
from utils.token_converter import get_tokenizer, text_to_token_ids, token_ids_to_text

In [4]:
def get_device():
    return "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
get_device()

'cuda'

### datset

In [ ]:
def format_input(entry):

    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately complates the request."
        f"\n\n## Instruction:\n{entry["instruction"]}"
    )
    input_text = (
        f"\n\n### Input:\n{entry["input"]}" if entry["input"] else " "

    )
    return instruction_text + input_text



# The custom collate function taked the data batch and makes the instructions and responses inside the batch of the same size.
def custom_collate_fn(
    batch,
    pad_token_id=50256,
    ignore_index=-100,
    allowed_max_length=None,
    device="cpu"
):
    input_batch, target_batch = zip(*batch)
    batch_max_length = max(item.shape[1] for item in input_batch)

    padded_inputs = []
    padded_targets = []
    
    for inputs, targets in zip(input_batch, target_batch):
        input = inputs.to(device)
        targets = targets.to(device)

        pad_length = batch_max_length - inputs.shape[1]
        if pad_length > 0:
            pad_tensor = torch.full((1, pad_length), pad_token_id, device=device)
            inputs = torch.cat([inputs, pad_tensor], dim=1)
            pad_targets = torch.full((1, pad_length), ignore_index, device=device)
            targets = torch.cat([targets, pad_targets], dim=1)

        if allowed_max_length is not None:
            inputs = inputs[:, :allowed_max_length]
            targets = targets[:, :allowed_max_length]

        padded_inputs.append(inputs)
        padded_targets.append(targets)

    input_tensors = torch.cat(padded_inputs, dim=0)
    target_tensors = torch.cat(padded_targets, dim=0)

    return input_tensors, target_tensors

In [7]:
class InstructionDataset(Dataset):

    def __init__(self, data, tokenizer, device="cpu"):
        self.data = data
        self.tokenizer = tokenizer
        self.device = device
        self.encoded_text = []
        for entry in data:
            instruction_plus_input = format_input(entry)
            response_text = f"\n\n### Response:\n{entry["output"]}"
            full_text = instruction_plus_input + response_text
            self.encoded_text.append(
                tokenizer.encode(full_text)
            )

    def __getitem__(self, index):

        encoded_text = self.encoded_text[index]
        # Return input and target tensors with correct shape and device
        inputs = torch.tensor(encoded_text[:-1], device=self.device).unsqueeze(0)
        targets = torch.tensor(encoded_text[1:], device=self.device).unsqueeze(0)
        return inputs, targets

    def __len__(self):
        return len(self.data)

In [8]:
def dataloaders(
    train_data,
    val_data,
    test_data,
    tokenizer,
    batch_size,
    device="cpu"
):
    train_dataset = InstructionDataset(train_data, tokenizer,device=device)
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        collate_fn=custom_collate_fn,
        shuffle=False,
        drop_last=False,
        num_workers=0
    )

    test_dataset = InstructionDataset(test_data, tokenizer,device=device)
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        collate_fn=custom_collate_fn,
        shuffle=False,
        drop_last=False,
        num_workers=0
    )

    val_dataset = InstructionDataset(val_data, tokenizer,device=device)
    val_dataloader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        collate_fn=custom_collate_fn,
        shuffle=False,
        drop_last=False,
        num_workers=0
    )

    return train_dataloader, test_dataloader, val_dataloader

In [9]:
def finetune_model(
                 model,
                 train_loader,
                 val_loader,
                 optimizer,
                 device,
                 num_epochs,
                 tokenizer,
                 val_data,
                 eval_freqs=5,
                 eval_iter=5,
                 ):

         
   
    start_time = time.time()
    train_losses, val_losses, token_seen = train_model(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            optimizer=optimizer,
            device=device,
            num_epochs=num_epochs,
            eval_freq=eval_freqs,
            eval_iter=eval_iter,
            start_context=format_input(val_data[0]),
            tokenizer=tokenizer
        )

    end_time = time.time()
    execution_time_in_minutes = (end_time - start_time) / 60
    print(f"training completed in {execution_time_in_minutes:.2f} minutes.")
    return model


## Finetunning

### Load pretrained model

In [ ]:
BASE_CONFIG = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "drop_rate": 0.0,       # Dropout rate
    "qkv_bias": True        # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}


CHOOSE_MODEL = "gpt2-small (355M)"
BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

gpt = GPTModel(BASE_CONFIG)
device = get_device()
tokenizer = get_tokenizer()
model_name = "GPT2-355M-pretrained"

model = load_model(model=gpt,
                   model_name=model_name,
                   device=device)

In [11]:
data = download_and_load_dataset("instruction-data.json")
train_data, test_data, val_data = partition_data(data)
train_dataloader, test_dataloader, val_dataloader = dataloaders(train_data, val_data, test_data, tokenizer=tokenizer,batch_size=16)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)
num_epochs = 30


Training data length: 935
Test data length: 110
Validation data length: 55


In [13]:
sft_model = finetune_model(
                 model=model,
                 train_loader=train_dataloader,
                 val_loader=val_dataloader,
                 optimizer=optimizer,
                 device=device,
                 num_epochs=num_epochs,
                 tokenizer=tokenizer,
                 val_data=val_data,
                 eval_freqs=5,
                 eval_iter=5,
                 )

Epoch 1: 100%|██████████| 59/59 [03:52<00:00,  3.94s/it, train_loss=0.635, val_loss=0.854]



[Epoch 1] Sample Generation:
Below is an instruction that describes a task. Write a response that appropriately complates the request.  ## Instruction: Convert the active sentence to passive: 'The chef cooks the meal every day.'   ### Response: The active sentence is passive.  ### Input: The chef cooked the meal every day.  ### Response: The active sentence is passive.  ### Input: The chef cooked the meal every


Epoch 2: 100%|██████████| 59/59 [03:33<00:00,  3.61s/it, train_loss=0.446, val_loss=0.752]



[Epoch 2] Sample Generation:
Below is an instruction that describes a task. Write a response that appropriately complates the request.  ## Instruction: Convert the active sentence to passive: 'The chef cooks the meal every day.'   ### Response: The chef cooked the meal every day.  ### Input: The meal was prepared by the chef.  ### Response: The meal was prepared by the chef.  ### Input: The chef


Epoch 3: 100%|██████████| 59/59 [03:44<00:00,  3.81s/it, train_loss=0.319, val_loss=0.796]


...


[Epoch 28] Sample Generation:
Below is an instruction that describes a task. Write a response that appropriately complates the request.  ## Instruction: Convert the active sentence to passive: 'The chef cooks the meal every day.'   ### Response: The meal every day is cooked by the chef.  ### Input: The chef cooks the meal every day.  ### Response: The meal every day is cooked by the chef.  ### Input


Epoch 29: 100%|██████████| 59/59 [03:42<00:00,  3.76s/it, train_loss=0.133, val_loss=0.944]



[Epoch 29] Sample Generation:
Below is an instruction that describes a task. Write a response that appropriately complates the request.  ## Instruction: Convert the active sentence to passive: 'The chef cooks the meal every day.'   ### Response: The meal every day is cooked by the chef.  ### Input: The chef cooks the meal every day.  ### Response: The meal every day is cooked by the chef.  ### Input


Epoch 30: 100%|██████████| 59/59 [03:28<00:00,  3.54s/it, train_loss=0.128, val_loss=0.943]



[Epoch 30] Sample Generation:
Below is an instruction that describes a task. Write a response that appropriately complates the request.  ## Instruction: Convert the active sentence to passive: 'The chef cooks the meal every day.'   ### Response: The meal every day is cooked by the chef.  ### Input: The chef cooks the meal every day.  ### Response: The meal every day is cooked by the chef.  ### Input
Model training has been completed.
training completed in 106.55 minutes.


In [ ]:
save_model(sft_model, "instruct-GPT2-355M-SFT.pth", optimizer=optimizer)

Pretrained model has been saved successfully at d:\software_3\Generative_models\Text_models\chat_gpt2\gpt_models\instruction_finetunned_model.pth


In [15]:
text = "Convert 45 kilometers to meters."

encoded_text =  text_to_token_ids(text, tokenizer).to(device)

idx = encoded_text
token_ids = generate(
        model=sft_model,
        idx=encoded_text,
        max_new_tokens=30,
        context_size=BASE_CONFIG["context_length"],
        temperature=0.0,
        top_k=None,
        eos_id=None
    )

print(token_ids_to_text(token_ids, tokenizer))

Convert 45 kilometers to meters. 

### Instruction:
What is the opposite of 'tall'? 

### Response:
The opposite of 'tall' is '
